In [2]:
# Import relevant packages
import pandas as pd

In [4]:
# read in data
traffic_data = pd.read_csv("traffic_data.csv")

In [5]:
# look into traffic rates

# rush hour rate: 8 percent of the daily traffic
traffic_data['rush_hour_rate'] = traffic_data['Average daily traffic counts Year_2015'] * 0.08


In [6]:
print(traffic_data)

     Route_ID  startMilepost  endMilepost  \
0           5         100.93       101.87   
1           5         101.87       103.17   
2           5         103.17       103.42   
3           5         103.42       104.81   
4           5         104.81       105.63   
..        ...            ...          ...   
219       520           9.60         9.72   
220       520           9.72        11.35   
221       520          11.35        12.05   
222       520          12.05        12.38   
223       520          12.38        12.83   

     Average daily traffic counts Year_2015  \
0                                     65000   
1                                     85000   
2                                    108000   
3                                    101000   
4                                    144000   
..                                      ...   
219                                   95000   
220                                   68000   
221                                 